In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
import

In [ ]:
df=pd.read_csv('/content/diabetes.csv')

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# Check for missing values
print(df.isnull().sum())  # Shows the count of missing values in each column


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [ ]:
X=df.iloc[:,:-1] # Select all rows, all columns except the last
y=df.iloc[:,-1] # Select all rows, only the last column (target)

In [ ]:
X_tensor=torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Converts [768] → [768,1]


print("Features shape:", X_tensor.shape)
print("Target shape:", y_tensor.shape)

Features shape: torch.Size([768, 8])
Target shape: torch.Size([768, 1])


In [ ]:
mean=X_tensor.mean(dim=0)
std=X_tensor.std(dim=0)
X_tensor=(X_tensor-mean)/std

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Validation set size:", X_val.shape)
print("Testing set size:", X_test.shape)


Training set size: torch.Size([491, 8])
Validation set size: torch.Size([123, 8])
Testing set size: torch.Size([154, 8])


In [ ]:
class DiabetesClassifier(nn.Module):
    def __init__(self):
        super(DiabetesClassifier, self).__init__()
        self.fc1 = nn.Linear(8, 4) # 8 input features → 4 neurons # Change input features from 3 → 8
        self.fc2 = nn.Linear(4, 1)# 4 neurons → 1 output

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

# Step 2: Create the Model
model = DiabetesClassifier()


In [ ]:
# It calculates how different our model's predictions are from the actual labels (0 or 1).
criterion = nn.BCELoss()

# accordinfg to the loss calculated the optimizer will update new values
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


criterion checks how wrong the predictions are.

optimizer adjusts the model to make better predictions in the next round.

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
  y_pred=model(X_tensor)

  loss=criterion(y_pred, y_tensor)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

  if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch 1/100, Loss: 0.6482870578765869
Epoch 2/100, Loss: 0.6470102667808533
Epoch 3/100, Loss: 0.6457396149635315
Epoch 4/100, Loss: 0.6444752216339111
Epoch 5/100, Loss: 0.6432172656059265
Epoch 6/100, Loss: 0.6419658064842224
Epoch 7/100, Loss: 0.6407210230827332
Epoch 8/100, Loss: 0.6394831538200378
Epoch 9/100, Loss: 0.6382521986961365
Epoch 10/100, Loss: 0.6370282769203186
Epoch [10/100], Loss: 0.6370
Epoch 11/100, Loss: 0.6358117461204529
Epoch 12/100, Loss: 0.634602427482605
Epoch 13/100, Loss: 0.6334004402160645
Epoch 14/100, Loss: 0.6322060823440552
Epoch 15/100, Loss: 0.6310192942619324
Epoch 16/100, Loss: 0.6298401951789856
Epoch 17/100, Loss: 0.6286688446998596
Epoch 18/100, Loss: 0.6275052428245544
Epoch 19/100, Loss: 0.6263495087623596
Epoch 20/100, Loss: 0.6252018809318542
Epoch [20/100], Loss: 0.6252
Epoch 21/100, Loss: 0.6240620017051697
Epoch 22/100, Loss: 0.6229302287101746
Epoch 23/100, Loss: 0.6218063235282898
Epoch 24/100, Loss: 0.620690643787384
Epoch 25/100, Los

model training completed


In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

with torch.no_grad():  # No gradient computation needed
    y_test_pred = model(X_test_tensor)  # Get predictions
    y_test_pred = (y_test_pred > 0.5).float()  # Convert probabilities to 0 or 1

    accuracy = (y_test_pred.eq(y_test_tensor).sum().item()) / y_test_tensor.shape[0]
    print(f'Accuracy: {accuracy / 100:.2f}')


Accuracy: 0.91


<ipython-input-35-824d797c6a2e>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
<ipython-input-35-824d797c6a2e>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)


In [ ]:
new_data = torch.tensor([[6, 148, 72, 35, 0, 33.6, 0.627, 50]], dtype=torch.float32)  # Example input
prediction = model(new_data)
prediction = (prediction > 0.5).float()  # Convert to 0 or 1
print(f'Predicted Class: {prediction.item()}')

Predicted Class: 1.0
